# Variational Quantum Classifier (VQC) Splitting

This notebook will explore the Qiskit implementation of a VQC, and initial attempts to split the VQC into parts to explain.


In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

### Imports

In [8]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import os

from utilities.dataset_utils import DiabetesData

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# qiskit imports
from qiskit import transpile
# algorithm
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.algorithms import VQR

# feature map
from qiskit.circuit.library import ZZFeatureMap

# ansatz
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import EfficientSU2

# optimizer
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.optimizers import SPSA
from qiskit_algorithms.optimizers import SLSQP
from qiskit_algorithms.optimizers import ADAM

# sampler
from qiskit.primitives import StatevectorSampler

#from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2 as Sampler

### VQC Overview

A Variational Quantum Classifier is a hybrid quantum-classical machine learning algorithm used for classification. It encodes the classical data using a feature map, which maps the data onto qubits to take advantage of quantum phenomena. The encoded data is processed through a variational quantum circuit (consists of parameterized gates). These parameters are iteratively optimised using classical algorithms to minimise a cost function (difference between predicted and true labels). After processing, quantum circuit is measured, and results are post processed to produce predictions. 
This hybrid approach allows the VQC allows the combination of the power of quantum mechanics with classical optimisation techniques, offering advantages for complex classification problems that would be challenging for a purely classical approach

In [9]:
# path to diabetes.csv
path = os.path.join(os.getcwd(), '..', '..', 'utilities', 'diabetes.csv')
# load dataset class
dataset = DiabetesData(path)

In [ ]:
# setup backend simulator
backend = AerSimulator(method='statevector')

In [ ]:
# feature map
feature_map = ZZFeatureMap(feature_dimension=dataset.get_num_features(), reps=1, entanglement='full')
feature_map = transpile(feature_map, backend=backend) # transpile to backend

In [ ]:
# ansatz
ansatz = EfficientSU2(num_qubits=dataset.get_num_features(), reps=2, entanglement='full')
ansatz = transpile(ansatz, backend=backend) # transpile to backend

In [12]:
# optimizer
optimizer = COBYLA(maxiter=300)

In [ ]:
# sampler
sampler = Sampler.from_backend(backend)

In [14]:
# create the VQC instance
vqc = VQC(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=optimizer,
    sampler=sampler
)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


In [15]:
# get training and testing data
X_train, X_test, y_train, y_test = dataset.preprocess_data()

In [16]:
# fit the model
vqc = vqc.fit(X_train, y_train)

In [17]:
train_score = vqc.score(X_train, y_train)
test_score = vqc.score(X_test, y_test)
train_score, test_score

(0.6499068901303539, 0.6536796536796536)

In [19]:
# predict
y_pred = vqc.predict(X_test)

In [20]:
# confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[131,  19],
       [ 60,  21]])

In [21]:
# accuracy score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.658008658008658

In [22]:
# classification report
report = classification_report(y_test, y_pred)
report

'              precision    recall  f1-score   support\n\n           0       0.69      0.87      0.77       150\n           1       0.53      0.26      0.35        81\n\n    accuracy                           0.66       231\n   macro avg       0.61      0.57      0.56       231\nweighted avg       0.63      0.66      0.62       231\n'

### Applying Explainability Techniques
##### **Pre-Prediciton**
1) **Feature Map**
    - Perturbation Analysis
    - Leave-One-Out (LOO) Analysis
    - Permutation Importance
    - SHAP (Feature Encoding)

    
##### **Post-Prediction**
1) **Ansatz (Variational Circuit)**
    - Parameter Sensitivity Analysis
    - Quantum Fisher Information (QFI)
    - Gradient-Based Analysis
    - Circuit Visualisation
    - SHAP (Parameters)
      - Applying importance scores to asses contribution to 
2) **Measurement + Post Processing**
    - SHAP (Predicitons)
    - ALE
    - Probability Attribution Analysis
3) **Cost**
    - Loss Contribution Analysis
    - Perturbation Sensitivity on Loss
    - Quantum-Classical Hybrid Loop Analysis